In [2]:
import pandas as pd
import csv
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline
from copy import deepcopy
import numpy as np
from sklearn.cluster import KMeans
import networkx as nx
import operator
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [3]:
data = pd.read_csv('cleanFeatures.csv')
data.head()
data.drop(['Unnamed: 0'], axis=1,inplace = True)

In [6]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
X = data.drop(['Target','ProteinID'],axis=1)
y = data['Target']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=2018)

In [13]:
X_train, y_train = make_classification(n_samples=1000, n_features=37,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=2,
                             random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [14]:
print(clf.feature_importances_)

[0.10358988 0.36997309 0.01070155 0.00438722 0.00251499 0.0209438
 0.02040495 0.01721657 0.00764198 0.01981717 0.02202737 0.02329363
 0.00589004 0.00680684 0.01253337 0.02331803 0.00998383 0.02993571
 0.00880603 0.01079705 0.00892219 0.00326208 0.01508154 0.0091504
 0.01626347 0.03743268 0.01730956 0.05338945 0.01424981 0.0113385
 0.02073894 0.00916768 0.00794465 0.01488356 0.01099797 0.0144245
 0.00485991]


In [16]:
predictions = clf.predict(X_test)

In [25]:
TP = 0
FP = 0
FN = 0
actual = y_test.tolist()
for i in range(len(predictions)):
    if(predictions[i] == actual[i]):
        TP += 1
    if((predictions[i] == 0) and (actual[i] == 1)):
        FP += 1
    if((predictions[i] == 1) and (actual[i] == 0)):
        FN += 1
        
F1 = (2*TP)/(2*TP+FP+FN)
TN = len(predictions) - TP - FP - FN
ACC = (TP+TN) / len(predictions)

In [26]:
F1

0.6612339930151339

In [27]:
ACC

0.49391304347826087